In [1]:
import mlagents
from mlagents_envs.environment import UnityEnvironment as UE
from mlagents_envs.envs.unity_parallel_env import UnityParallelEnv as UPZBE
from PPO_Distillation.DistilledPPOAgent import DistilledPPO
from PPO_Distillation.Trajectories import ExperienceBuffer
from PPO_Distillation.Hyperparameters import HYPERPARAMS as params
import numpy as np
import torch
import cv2

c:\Users\rullo\anaconda3\envs\ml_agents\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
env = UE(file_name="DroneFlightv1", seed=1, side_channels=[], no_graphics_monitor=True, no_graphics=True)
env = UPZBE(env)

In [3]:
agents = [agent for agent in env.agents]
print(agents)

['Drone?team=0?agent_id=0', 'Drone?team=0?agent_id=1']


In [4]:
obs = env.reset()
print(obs[agents[0]][2])
print(env.action_space(agents[0]).shape)
print(env.observation_space(agents[0])[1].shape)
print(env.observation_space(agents[0])[2].shape)

[ 0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.    0.
  0.   10.    0.   -8.95 20.   25.77  0.   10.    0.   -8.95 20.   25.77]
(6,)
(1, 84, 84)
(24,)


In [5]:
Buffer = ExperienceBuffer(env.observation_space(agents[0])[1].shape, env.observation_space(agents[0])[2].shape,env.action_space(agents[0]).shape, params['ppo_distilled'])

In [6]:
def relocate_agents():
    return [agent for agent in env.agents]

In [7]:
# # for s in range(1, params.seed_episodes + 1):
# for s in range(1, 2):
#     obs, done, t = env.reset(), [False for _ in env.agents], 0
#     while not all(done) or t < params['ppo_distilled'].n_steps_random_exploration:
#         actions = {}
#         agents = relocate_agents()
#         for agent in agents:
#             actions[agent] = env.action_space(agent).sample()
#         obs, reward, done, _ = env.step(actions)
#         for agent in agents:
#             if agent not in obs.keys():
#                 continue
#             if isinstance(obs[agent], list):
#                 Buffer.add(np.array(obs[agent][1]), np.array(obs[agent][2]), actions[agent], reward[agent], done[agent], log_prob=0, ext_value=0, int_value=0)
#             else:
#                 Buffer.add(np.array(obs[agent]['observation'][1]),np.array(obs[agent]['observation'][2]), actions[agent], reward[agent], done[agent], log_prob=0, ext_value=0, int_value=0)
#         t += 1
#         done = [done[agent] for agent in agents if agent in done.keys()]
#     print(f'Finished episode {s}')

# print("Finished Rnd Exploration")
# env.close()

In [8]:
env.close()
env = UE(file_name="DroneFlightv1", seed=1, side_channels=[], no_graphics_monitor=True, no_graphics=True)
env = UPZBE(env)
agents = relocate_agents()

In [9]:
brain = DistilledPPO(env.observation_space(agents[0])[1].shape, env.observation_space(agents[0])[2].shape, env.action_space(agents[0]).shape, params['ppo_distilled'])

Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.


In [10]:
agents = relocate_agents()
metrics = brain.train(1, Buffer)

tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]], device='cuda:0')


c:\Users\rullo\anaconda3\envs\ml_agents\lib\site-packages\transformers\image_processing_utils.py:41: UserWarning: The following named arguments are not valid for `ViTImageProcessor.preprocess` and were ignored: 'return_tesors'
  return self.preprocess(images, **kwargs)


ValueError: mean must have 1 elements if it is an iterable, got 3